In [1]:
#%pip install ase==3.22.1
#%pip install numpy==1.24.4
#%pip install dscribe==2.5.0
#%pip install scipy

# Compare between each substances Euclid Distance and Cosine Similarity
# Brownmillerite perovskite structure (POSCAR INPUT) 
Structure sponsured by Takatsuka san

In [ ]:
from ase.io import read
from ase import Atoms

atoms = read("BM_POSCAR")
atoms_mattersim = read("MatterSim_POSCAR")
atoms_CHGnet = read("CHGNet_POSCAR")


In [5]:
#Parameters setting
r_cut=5
n_max=2
l_max=0

In [7]:
species = list(set(atoms.get_chemical_symbols()))
print(species)

['Sr', 'Co', 'O']


In [ ]:
import numpy as np
from dscribe.descriptors import SOAP
from ase import Atoms
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean


species = list(set(atoms.get_chemical_symbols()))

# Create SOAP descriptor with averaging mode
soap = SOAP(
    species=species,
    periodic=True,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner"  # Proper average mode
)

# Create SOAP descriptors (averaged over atoms)
vec_bm = soap.create(atoms, positions=[atom.index for atom in atoms])

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x109a80400>>
Traceback (most recent call last):
  File "/Users/kongkitimanon2024/.pyenv/versions/miniforge3-22.11.1-4/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


ValueError: The following atomic numbers are not defined for this descriptor: {14}

# Create average vector for each substance

In [5]:
#local descriptor into a global descriptor is simply by taking the average over all sites
average_soap = SOAP(
    species=species,
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    average="inner",
    sparse=False
)
#Sio2_quartz
average_si_o2_quartz = average_soap.create(si_o2_quartz)
print("Average SOAP si_o2_quartz",average_si_o2_quartz.shape)

#Sio2_cristobaliter
average_si_o2_cristobaliter = average_soap.create(si_o2_cristobaliter)
print("Average SOAP si_o2_cristobaliter",average_si_o2_cristobaliter.shape)

#GaAs_Wurtzite
average_ga_as_wurtzite = average_soap.create(ga_as_wurtzite)
print("Average SOAP ga_as_wurtzite",average_ga_as_wurtzite.shape)

#GaAs_Zinc_Blend
average_ga_as_zinc_blend = average_soap.create(ga_as_zinc_blend)
print("Average SOAP ga_as_zinc_blend",average_ga_as_zinc_blend.shape)

Average SOAP si_o2_quartz (36,)
Average SOAP si_o2_cristobaliter (36,)
Average SOAP ga_as_wurtzite (36,)
Average SOAP ga_as_zinc_blend (36,)


# Euclid Distance calculation

In [6]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

molecules_si_o2 = np.vstack([average_si_o2_quartz, average_si_o2_cristobaliter])
distance_si_o2 = squareform(pdist(molecules_si_o2))
print("Distance matrix between SiO2: ")
print(distance_si_o2)
#Euclid Distance calculate
print("Euclid distance between SiO2: ",np.linalg.norm(distance_si_o2))
print("")

molecules_ga_as = np.vstack([average_ga_as_wurtzite, average_ga_as_zinc_blend])
distance_ga_as = squareform(pdist(molecules_ga_as))
print("Distance matrix between GaAs: ")
print(distance_ga_as)
#Euclid Distance calculate
print("Euclid distance between GaAs: ",np.linalg.norm(distance_ga_as))
print("")


Distance matrix between SiO2: 
[[0.         5.12437079]
 [5.12437079 0.        ]]
Euclid distance between SiO2:  7.24695466967479

Distance matrix between GaAs: 
[[ 0.        14.4982268]
 [14.4982268  0.       ]]
Euclid distance between GaAs:  20.50358896765264



# Cosine Similarity Calculation

The similarity can take values between -1 and +1. Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity. 

cos(angle)= dot product between vectors /(L2 norm of vector1)* (L2 norm of vector2)

For example:
- When two vectors have the same orientation, the angle between them is 0, and the cosine similarity is 1.
- Perpendicular vectors have a 90-degree angle between them and a cosine similarity of 0.
- Opposite vectors have an angle of 180 degrees between them and a cosine similarity of -1.

Ref: https://www.learndatasci.com/glossary/cosine-similarity/

In [12]:
import math

def cos_sim(p, q):
    return np.dot(p, q) / (np.linalg.norm(p) * np.linalg.norm(q))

cos_sim_si_o2 =cos_sim(average_si_o2_quartz,average_si_o2_cristobaliter)
print("cos_sim_si_o2: ",cos_sim_si_o2)

cos_sim_ga_as =cos_sim(average_ga_as_wurtzite, average_ga_as_zinc_blend)
print("cos_sim_ga_as: ",cos_sim_ga_as)


cos_sim_si_o2:  0.99999100536915
cos_sim_ga_as:  0.9988064907593264
